In [1]:
import os
import networkx as nx
import numpy as np
import pandas as pd
import math
import pyflagser
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from operator import add
import statistics
import sys
import pickle
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import scikitplot as skplt
import matplotlib.pyplot as plt

In [2]:
G=nx.read_gml("123 Bus/123busEx.gml")
A = nx.to_numpy_array(G)
N=list(G.nodes)
E=list(G.edges)
with open('123 Bus/fullanomalydataset.pkl', 'rb') as g:
    P0Data = pickle.load(g)

In [3]:
Class=[]
for i in range(500):
    Class.append(P0Data[i]["Anomalous"])
print(len(Class))
i=0
l=0
m=0
while i < len(Class):
    if Class[i] == 'No':
        l=l+1
        Class[i] = 0
    if Class[i] == 'Yes':
        m=m+1
        Class[i] = 1
    i += 1
#print(Class)
print(l)
print(m)

500
251
249


In [4]:
def Average(lst):
    length=sum(1 for i in lst if i)
    return sum(lst) / length

In [52]:
N_Senario=len(P0Data)
print(N_Senario)

500


# Single Persistence

In [5]:
def TimeSeries_Fe(TS_array, F_voltage):
    betti_0=[]
    for k in range(len(TS_array)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_array[k]
        for y in Voltage:
            AverageVoltage.append(Average(list(y)))
        #AverageVoltage = [i * 10 for i in AverageVoltage]
        for p in range(len(F_voltage)):
            n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                fec.append(0)
            else:
                b=A[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                fec.append(x[0])
            n_active.clear()
        betti_0.append(fec)
    return betti_0

In [431]:
#F_voltage=[105,104,103,102,100,99,98,97,96,95,94,93,92,90,88]
#F_voltage=[1.1,1.0,0.90,0.89,0.88]
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.95,0.90,0.89,0.88,0.87,0.86,0.85,0.7]
betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)

In [6]:
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.99,0.98,0.97,0.96,0.95,0.94,0.92,0.90,0.89,0.88,0.87,0.86,0.85]
Betti_0=[]
N_Senario=len(P0Data)
#N_Senario=5
for i in range(N_Senario):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(N_Senario-1)), end='', flush=True)
    TimeSeries_Voltage=P0Data[i]["TimeSeries_Voltage"]
    betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)
    #print(betti)
    #Betti=Make_Sum(betti)
    #print(Betti)
    Betti_0.append(betti)
    #Betti_0.append(Betti)

Processing file 499 (100%)

In [8]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [21]:
from sklearn.model_selection import KFold
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)

In [22]:
X = Betti_0.copy()
y = Class.copy()
X=np.array(X)
y=np.array(y)

In [23]:
num_samples = 500
num_timesteps = 24
num_features = len(F_voltage)
num_classes = 2

In [24]:
loss_per_fold=[]
acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    # Define the LSTM model
    model = Sequential([
        LSTM(10, activation='tanh',recurrent_activation='sigmoid',dropout=0.0,input_shape=(num_timesteps, num_features)),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X[train], y[train], epochs=100, batch_size=32)
    # Evaluate the model
    #loss, accuracy = model.evaluate(X[test], y[test])
    #print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
    scores = model.evaluate(X[test], y[test])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

Epoch 1/100


2024-04-16 12:39:45.322262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 12:39:45.417250: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 3/15 [=====>........................] - ETA: 0s - loss: 0.7311 - accuracy: 0.5521

2024-04-16 12:39:45.545508: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 19ms/step - loss: 0.7406 - accuracy: 0.4933
Epoch 2/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6974 - accuracy: 0.5089
Epoch 3/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6901 - accuracy: 0.5289
Epoch 4/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6877 - accuracy: 0.5311
Epoch 5/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6854 - accuracy: 0.5378
Epoch 6/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6820 - accuracy: 0.5422
Epoch 7/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6795 - accuracy: 0.5333
Epoch 8/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6733 - accuracy: 0.5556
Epoch 9/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6690 - accuracy: 0.5911
Epoch 10/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6686 - accuracy: 0.6022
Epoch 11/100
15/15 [

...

 3/15 [=====>........................] - ETA: 0s - loss: 0.7080 - accuracy: 0.4792

2024-04-16 12:41:39.990307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 25ms/step - loss: 0.7208 - accuracy: 0.4689
Epoch 2/100
15/15 [==============================] - 0s 8ms/step - loss: 0.7098 - accuracy: 0.4933
Epoch 3/100
15/15 [==============================] - 0s 8ms/step - loss: 0.7039 - accuracy: 0.5044
Epoch 4/100
15/15 [==============================] - 0s 8ms/step - loss: 0.7016 - accuracy: 0.4822
Epoch 5/100
15/15 [==============================] - 0s 7ms/step - loss: 0.6997 - accuracy: 0.4178
Epoch 6/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6943 - accuracy: 0.5067
Epoch 7/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6935 - accuracy: 0.5178
Epoch 8/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6914 - accuracy: 0.5178
Epoch 9/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6906 - accuracy: 0.5311
Epoch 10/100
15/15 [==============================] - 0s 8ms/step - loss: 0.6896 - accuracy: 0.5333
Epoch 11/100
15/15 [

15/15 [==============================] - 0s 8ms/step - loss: 0.4250 - accuracy: 0.8111
Epoch 84/100
15/15 [==============================] - 0s 8ms/step - loss: 0.4814 - accuracy: 0.7800
Epoch 85/100
15/15 [==============================] - 0s 8ms/step - loss: 0.4636 - accuracy: 0.7622
Epoch 86/100
15/15 [==============================] - 0s 8ms/step - loss: 0.4436 - accuracy: 0.7867
Epoch 87/100
15/15 [==============================] - 0s 7ms/step - loss: 0.4162 - accuracy: 0.8044
Epoch 88/100
15/15 [==============================] - 0s 7ms/step - loss: 0.4365 - accuracy: 0.7933
Epoch 89/100
15/15 [==============================] - 0s 7ms/step - loss: 0.4237 - accuracy: 0.8022
Epoch 90/100
15/15 [==============================] - 0s 7ms/step - loss: 0.4078 - accuracy: 0.8156
Epoch 91/100
15/15 [==============================] - 0s 7ms/step - loss: 0.3966 - accuracy: 0.8289
Epoch 92/100
15/15 [==============================] - 0s 8ms/step - loss: 0.4363 - accuracy: 0.7956
Epoch 93/100


2024-04-16 12:41:52.114351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 12:41:52.154510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [25]:
print(acc_per_fold)

[72.00000286102295, 77.99999713897705, 80.0000011920929, 89.99999761581421, 77.99999713897705, 80.0000011920929, 75.99999904632568, 83.99999737739563, 81.99999928474426, 83.99999737739563]


In [26]:
import statistics
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using LSTM", mean_value)
print("Standard Deviation:", std_dev)

Mean using LSTM 80.39999902248383
Standard Deviation: 4.971026051642439


# Multi Persistence

In [27]:
def TimeSeries_Fe_MP(TS_voltage, TS_branchFlow, F_voltage,F_Flow):
    betti_0=[]
    for k in range(len(TS_voltage)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_voltage[k]
        for y in Voltage:
            AverageVoltage.append(Average(list(y)))
        #AverageVoltage = [i * 100 for i in AverageVoltage]
        BranchFlow=[]
        Branch_Flow=TS_branchFlow[k]
        for j  in range(len(Branch_Flow)):
            BranchFlow.append(Branch_Flow[j][0])

        for p in range(len(F_voltage)):
            Active_node_v=np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            for q in range(len(F_Flow)):
                #if AverageVoltage[p]> F_voltage[p] and BranchFlow[q]> F_Flow[q]:
                #n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
                n_active=Active_node_v.copy()
                #print(n_active)
                G = nx.DiGraph()
                G.add_nodes_from(n_active)
                indices = np.where(np.array(BranchFlow) > F_Flow[q])[0].tolist()
                for s in indices:
                    a=int(N.index(E[s][0]))
                    b=int(N.index(E[s][1]))
                    if a in n_active and b in n_active:
                        G.add_edge(a, b)
                    #n_active.append(int(N.index(E[s][0])))
                    #n_active.append(int(N.index(E[s][1])))
                #Active_node=np.unique(n_active)
                #print(G.edges())
                if (len(n_active)==0):
                    fec.append(0)
                else:
                    #b=A[Active_node,:][:,Active_node]
                    Adj = nx.to_numpy_array(G)
                    my_flag=pyflagser.flagser_unweighted(Adj, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                    x = my_flag["betti"]
                    fec.append(x[0])
                n_active.clear()
            Active_node_v.clear()
        betti_0.append(fec)
    return betti_0




In [28]:
F_Flow=[100,90,75,50,25,10,5,0]
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.99,0.98,0.97,0.96,0.95,0.94,0.92,0.90,0.89,0.88,0.87,0.86,0.85]
Betti_0=[]
#N_Senario=len(P0Data)
N_Senario=300
for i in range(N_Senario):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(N_Senario-1)), end='', flush=True)
    TimeSeries_Voltage=P0Data[i]["TimeSeries_Voltage"]
    TimeSeries_Branch_Flow=P0Data[i]["BranchFlow"]
    betti=TimeSeries_Fe_MP(TimeSeries_Voltage,TimeSeries_Branch_Flow, F_voltage,F_Flow)
    #print(betti)
    #Betti=Make_Sum(betti)
    #print(Betti)
    Betti_0.append(betti)
    #Betti_0.append(Betti)

Processing file 299 (100%)

In [29]:
Class=[]
for i in range(300):
    Class.append(P0Data[i]["Anomalous"])
print(len(Class))
i=0
l=0
m=0
while i < len(Class):
    if Class[i] == 'No':
        l=l+1
        Class[i] = 0
    if Class[i] == 'Yes':
        m=m+1
        Class[i] = 1
    i += 1
#print(Class)
print(l)
print(m)

300
148
152


In [30]:
from sklearn.model_selection import KFold
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)
X = Betti_0.copy()
y = Class.copy()
X=np.array(X)
y=np.array(y)

In [39]:
from sklearn.preprocessing import MinMaxScaler

num_samples = 300
num_timesteps = 24
num_features = len(F_voltage)*len(F_Flow)
num_classes = 2
loss_per_fold=[]
acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    # Define the LSTM model
    model = Sequential([
        LSTM(10, activation='tanh',recurrent_activation='sigmoid',dropout=0.0,input_shape=(num_timesteps, num_features)),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    X0=[]
    for i in range(300):
        
        scaler = MinMaxScaler()

# Fit scaler to data and transform data
        X0.append(scaler.fit_transform(X[i]))
    X0=np.array(X0)
    #print(X0[0])
    # Train the model
    model.fit(X0[train], y[train], epochs=50, batch_size=32)
    # Evaluate the model
    #loss, accuracy = model.evaluate(X[test], y[test])
    #print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
    scores = model.evaluate(X0[test], y[test])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

Epoch 1/50


2024-04-16 23:46:54.337522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:46:54.416805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/9 [==>...........................] - ETA: 5s - loss: 0.7258 - accuracy: 0.5312

2024-04-16 23:46:54.587781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 49ms/step - loss: 0.6830 - accuracy: 0.5296
Epoch 2/50
9/9 [==============================] - 0s 24ms/step - loss: 0.6663 - accuracy: 0.5593
Epoch 3/50
9/9 [==============================] - 0s 13ms/step - loss: 0.6481 - accuracy: 0.6037
Epoch 4/50
9/9 [==============================] - 0s 13ms/step - loss: 0.6210 - accuracy: 0.6444
Epoch 5/50
9/9 [==============================] - 0s 10ms/step - loss: 0.5808 - accuracy: 0.6778
Epoch 6/50
9/9 [==============================] - 0s 8ms/step - loss: 0.5352 - accuracy: 0.7259
Epoch 7/50
9/9 [==============================] - 0s 9ms/step - loss: 0.5128 - accuracy: 0.7333
Epoch 8/50
9/9 [==============================] - 0s 10ms/step - loss: 0.4631 - accuracy: 0.7778
Epoch 9/50
9/9 [==============================] - 0s 9ms/step - loss: 0.4282 - accuracy: 0.8259
Epoch 10/50
9/9 [==============================] - 0s 8ms/step - loss: 0.4070 - accuracy: 0.8296
Epoch 11/50
9/9 [===========================

...

2024-04-16 23:47:44.556747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:47:44.591512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Score for fold 9: loss of 0.3759712874889374; accuracy of 86.66666746139526%
Epoch 1/50


2024-04-16 23:47:45.194351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:47:45.272653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:47:45.452182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 49ms/step - loss: 0.7300 - accuracy: 0.5074
Epoch 2/50
9/9 [==============================] - 0s 18ms/step - loss: 0.7005 - accuracy: 0.5037
Epoch 3/50
9/9 [==============================] - 0s 16ms/step - loss: 0.6927 - accuracy: 0.5148
Epoch 4/50
9/9 [==============================] - 0s 14ms/step - loss: 0.6872 - accuracy: 0.5333
Epoch 5/50
9/9 [==============================] - 0s 9ms/step - loss: 0.6828 - accuracy: 0.5593
Epoch 6/50
9/9 [==============================] - 0s 17ms/step - loss: 0.6751 - accuracy: 0.6333
Epoch 7/50
9/9 [==============================] - 0s 9ms/step - loss: 0.6637 - accuracy: 0.6556
Epoch 8/50
9/9 [==============================] - 0s 9ms/step - loss: 0.6406 - accuracy: 0.6778
Epoch 9/50
9/9 [==============================] - 0s 9ms/step - loss: 0.6019 - accuracy: 0.6889
Epoch 10/50
9/9 [==============================] - 0s 8ms/step - loss: 0.5620 - accuracy: 0.7296
Epoch 11/50
9/9 [============================

2024-04-16 23:47:50.157211: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:47:50.197882: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [37]:
import statistics
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using LSTM", mean_value)
print("Standard Deviation:", std_dev)

Mean using LSTM 90.66666543483734
Standard Deviation: 5.397758537561357
